In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

# Kelas untuk preprocessing data
class DataPreprocessor:
    def __init__(self, df):
        self.df = df

    def encode_categorical(self):
        label_encoders = {}
        for col in ['limit_cat', 'geo_cat']:
            le = LabelEncoder()
            self.df[col] = le.fit_transform(self.df[col])
            label_encoders[col] = le
        return label_encoders

    def normalize_features(self):
        scaler = StandardScaler()
        self.df[['transaction_dollar_amount', 'time_diff', 'distance']] = scaler.fit_transform(
            self.df[['transaction_dollar_amount', 'time_diff', 'distance']]
        )
        return scaler

    def process(self):
        self.encode_categorical()
        scaler = self.normalize_features()
        return self.df, scaler

# Kelas untuk membangun model Neural Network
class NeuralNetworkModel:
    def __init__(self, input_dim):
        self.model = self.build_model(input_dim)

    def build_model(self, input_dim):
        model = Sequential([
            Dense(32, activation='relu', input_shape=(input_dim,)),
            Dropout(0.3),
            Dense(16, activation='relu'),
            Dropout(0.2),
            Dense(1, activation='sigmoid')
        ])
        model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
        return model

    def train(self, X_train, y_train, X_val, y_val, epochs=10, batch_size=32):
        self.model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=epochs, batch_size=batch_size)

    def predict(self, X):
        return self.model.predict(X)

# Kelas utama yang mengelola proses deteksi fraud
class FraudDetector:
    def __init__(self, df):
        self.df = df
        self.preprocessor = DataPreprocessor(df)
        self.model = None

    def prepare_data(self):
        self.df, scaler = self.preprocessor.process()
        X = self.df.drop(columns=['fraud_status'])
        y = (self.df['fraud_status'] == 'fraud').astype(int)  # Convert label ke binary
        X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
        return X_train, X_val, y_train, y_val

    def train_model(self):
        X_train, X_val, y_train, y_val = self.prepare_data()
        self.model = NeuralNetworkModel(input_dim=X_train.shape[1])
        self.model.train(X_train, y_train, X_val, y_val)

    def detect_fraud(self, new_data):
        return self.model.predict(new_data)

# Contoh penggunaan
df = pd.DataFrame({
    'transaction_dollar_amount': [17.99, 12.09, 78.21, 74.41, 54.89],
    'time_diff': [-7642455.0, -2527299.0, -6508550.0, -2534699.0, -1785659.0],
    'distance': [11.969568, 15.511210, 9.404226, 9.556419, 8.157130],
    'limit_cat': ['very_low', 'low', 'very_high', 'medium', 'very_low'],
    'geo_cat': ['normal', 'normal', 'normal', 'normal', 'normal'],
    'fraud_status': ['not_fraud', 'not_fraud', 'not_fraud', 'not_fraud', 'not_fraud']
})

detector = FraudDetector(df)
detector.train_model()


ModuleNotFoundError: No module named 'tensorflow'